In [5]:
import psycopg2
import pandas as pd
from psycopg2 import extras

In [6]:
# Database connection details 
db_name = "postgres"
db_user = "postgres"
db_password = "Harsh@1234" 
db_host = "127.0.0.1"  # Assuming PostgreSQL is running on the docker machine
# Table name
table_name = "sales_data"

def process_data():
    """
    Processes data from the sales database table and returns a tuple containing:
    - A dictionary where the keys are transaction IDs, and values are the total profit for each transaction.
    - A dictionary where the keys are product IDs, and values are the total profit for each product.
    - The product IDs of the 2 top-selling products.
    """
    # Connect to the database
    try:
        
        connection = psycopg2.connect(user=db_user, password=db_password, host=db_host, port=5432, dbname=db_name)
        cursor = connection.cursor()
    
        # Query to select all data from the table
        select_query = f"SELECT * FROM {table_name};"
        cursor.execute(select_query)
        records = cursor.fetchall()
    
        # Create a DataFrame from the database records
        df = pd.DataFrame(records, columns=['TransactionID', 'ProductID', 'Quantity', 'SalePrice', 'PurchasePrice'])
    
        # Calculate the total profit for each transaction
        df['TotalProfit'] = df['Quantity'] * (df['SalePrice'] - df['PurchasePrice'])
    
        # Create a dictionary for transaction profits
        transaction_profits = df.set_index('TransactionID')['TotalProfit'].to_dict()
    
        # Calculate the total profit for each product
        product_profits = df.groupby('ProductID')['TotalProfit'].sum().to_dict()
    
        # Identify the top 2 selling products based on the total quantity
        top_selling_products = df.groupby('ProductID')['Quantity'].sum().nlargest(2).index.tolist()
    
        # Close the cursor and connection
        cursor.close()
        connection.close()
    
        # Return the results as a tuple
        return transaction_profits, product_profits, top_selling_products
    except (Exception, psycopg2.Error) as error:
        print("Error processing data:", error)
        raise  # Re-raise the exception for proper handling
        
# Example usage of the function
try:
    transaction_profits, product_profits, top_selling_products = process_data()
except Exception as e:
    print(f"An error occurred: {e}")

In [7]:
#the keys(index) are product IDs, and values are the total profit for each product.
product_profits_df = pd.DataFrame.from_dict(product_profits, orient='index', columns=['Total Profit'])
print(product_profits_df)

     Total Profit
1100       137.70
1101       224.32
1102        37.05
1103       488.47
1104        67.48
1105       245.54
1106        42.37
1107        84.89
1108        75.87
1109       201.58
1110      1003.12
1111        81.42
1112       479.26
1113       897.28
1114       170.74
1115       575.71
1116       263.49
1117       734.12
1118       153.74
1119  55852473.72
1120       888.22
1121       775.20
1122      1584.55
1123       632.49
1124       779.51
1125      1624.59
1126       839.89
1127       269.66
1128      1299.84
1129      2261.08
1130       640.40
1131       844.33
1132      1446.94
1133       778.18
1134      1756.16
1135      2276.95
1136      1709.81
1137       134.26
1138      1359.72
1139      1113.75
1140      2793.40
1141      1558.24
1142      1801.36
1143       649.99
1144        49.77
1145      1383.06
1146      1574.40
1147       157.29
1148       212.96
1149      1063.44
1150        32.20


In [8]:
#product IDs of the 2 top-selling products.
top_selling_products_df = pd.DataFrame(top_selling_products, columns=['Product ID'])
print(top_selling_products_df)


   Product ID
0        1119
1        1128


In [ ]:
#i also see there is an outlier on productID 1119 which has profit 55852473.72, whihch varies lot copared to other product id